###Ingest results.json file

In [0]:
dbutils.widgets.text('p_data_source','')
v_data_source = dbutils.widgets.get('p_data_source')

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

#####Step 1 - Read the json file using spark datafram reader API

In [0]:
from pyspark.sql.types import StructType,StructField,IntegerType,FloatType,StringType

In [0]:
results_schema = StructType(fields = [StructField('resultId',IntegerType(),False),
                                      StructField('raceId',IntegerType(),True),
                                      StructField('driverId',IntegerType(),True),
                                      StructField('constructorId',IntegerType(),True),
                                      StructField('number',IntegerType(),True),
                                      StructField('grid',IntegerType(),True),
                                      StructField('position',IntegerType(),True),
                                      StructField('positionText',StringType(),True),
                                      StructField('positionOrder',IntegerType(),True),
                                      StructField('points',FloatType(),True),
                                      StructField('laps',IntegerType(),True),
                                      StructField('time',StringType(),True),
                                      StructField('milliseconds',IntegerType(),True),
                                      StructField('fastestLap',IntegerType(),True),
                                      StructField('rank',IntegerType(),True),
                                      StructField('fastestLapTime',StringType(),True),
                                      StructField('fastestLapSpeed',FloatType(),True),
                                      StructField('statusId',StringType(),True),
                                      ])

In [0]:
results_df = spark.read.json(f"{raw_folder_path}/results.json",schema = results_schema)

In [0]:
results_df.show(10)

+--------+------+--------+-------------+------+----+--------+------------+-------------+------+----+-----------+------------+----------+----+--------------+---------------+--------+
|resultId|raceId|driverId|constructorId|number|grid|position|positionText|positionOrder|points|laps|       time|milliseconds|fastestLap|rank|fastestLapTime|fastestLapSpeed|statusId|
+--------+------+--------+-------------+------+----+--------+------------+-------------+------+----+-----------+------------+----------+----+--------------+---------------+--------+
|       1|    18|       1|            1|    22|   1|       1|           1|            1|  10.0|  58|1:34:50.616|     5690616|        39|   2|      1:27.452|          218.3|       1|
|       2|    18|       2|            2|     3|   5|       2|           2|            2|   8.0|  58|     +5.478|     5696094|        41|   3|      1:27.739|        217.586|       1|
|       3|    18|       3|            3|     7|   7|       3|           3|            3|  

#####Step 2 - Rename the columns and add new columns

In [0]:
from pyspark.sql.functions import current_timestamp,lit

In [0]:
results_with_columns_df = results_df.withColumnRenamed('resultId','result_id') \
                                    .withColumnRenamed('raceId','race_id') \
                                    .withColumnRenamed('driverId','driver_id') \
                                    .withColumnRenamed('constructorId','constructor_id') \
                                    .withColumnRenamed('positionText','position_text') \
                                    .withColumnRenamed('positionOrder','position_order') \
                                    .withColumnRenamed('fastestLap','fastest_lap') \
                                    .withColumnRenamed('fastestLapTime','fastest_lap_time') \
                                    .withColumnRenamed('fastestLapSpeed','fastest_lap_speed') \
                                    .withColumn('ingestion_date',current_timestamp()) \
                                    .withColumn('data_source',lit(v_data_source))

In [0]:
results_with_columns_df.show(10)

+---------+-------+---------+--------------+------+----+--------+-------------+--------------+------+----+-----------+------------+-----------+----+----------------+-----------------+--------+--------------------+-----------+
|result_id|race_id|driver_id|constructor_id|number|grid|position|position_text|position_order|points|laps|       time|milliseconds|fastest_lap|rank|fastest_lap_time|fastest_lap_speed|statusId|      ingestion_date|data_source|
+---------+-------+---------+--------------+------+----+--------+-------------+--------------+------+----+-----------+------------+-----------+----+----------------+-----------------+--------+--------------------+-----------+
|        1|     18|        1|             1|    22|   1|       1|            1|             1|  10.0|  58|1:34:50.616|     5690616|         39|   2|        1:27.452|            218.3|       1|2023-09-30 18:03:...| Ergast API|
|        2|     18|        2|             2|     3|   5|       2|            2|             2|  

#####Step 3 - Drop the unwanted column

In [0]:
from pyspark.sql.functions import col

In [0]:
results_final_df = results_with_columns_df.drop(col('statusId'))

In [0]:
results_final_df.show(10)

+---------+-------+---------+--------------+------+----+--------+-------------+--------------+------+----+-----------+------------+-----------+----+----------------+-----------------+--------------------+-----------+
|result_id|race_id|driver_id|constructor_id|number|grid|position|position_text|position_order|points|laps|       time|milliseconds|fastest_lap|rank|fastest_lap_time|fastest_lap_speed|      ingestion_date|data_source|
+---------+-------+---------+--------------+------+----+--------+-------------+--------------+------+----+-----------+------------+-----------+----+----------------+-----------------+--------------------+-----------+
|        1|     18|        1|             1|    22|   1|       1|            1|             1|  10.0|  58|1:34:50.616|     5690616|         39|   2|        1:27.452|            218.3|2023-09-30 18:03:...| Ergast API|
|        2|     18|        2|             2|     3|   5|       2|            2|             2|   8.0|  58|     +5.478|     5696094| 

#####Step 4 - Write the output to processed container in parquet format

In [0]:
results_final_df.write.mode('overwrite').partitionBy('race_id').parquet(f"{processed_folder_path}/results")

In [0]:
dbutils.notebook.exit('Success')